<a href="https://colab.research.google.com/github/xKDR/Julia-Workshop/blob/main/MachineLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <img src="https://github.com/JuliaLang/julia-logo-graphics/raw/master/images/julia-logo-color.png" height="100" /> _Colab Notebook Template_

## Instructions
1. Work on a copy of this notebook: _File_ > _Save a copy in Drive_ (you will need a Google account). Alternatively, you can download the notebook using _File_ > _Download .ipynb_, then upload it to [Colab](https://colab.research.google.com/).
2. If you need a GPU: _Runtime_ > _Change runtime type_ > _Harware accelerator_ = _GPU_.
3. Execute the following cell (click on it and press Ctrl+Enter) to install Julia, IJulia and other packages (if needed, update `JULIA_VERSION` and the other parameters). This takes a couple of minutes.
4. Reload this page (press Ctrl+R, or ⌘+R, or the F5 key) and continue to the next section.

_Notes_:
* If your Colab Runtime gets reset (e.g., due to inactivity), repeat steps 2, 3 and 4.
* After installation, if you want to change the Julia version or activate/deactivate the GPU, you will need to reset the Runtime: _Runtime_ > _Factory reset runtime_ and repeat steps 3 and 4.

In [ ]:
%%shell
set -e

#---------------------------------------------------#
JULIA_VERSION="1.11.2" # any version ≥ 0.7.0
JULIA_PACKAGES="IJulia BenchmarkTools"
JULIA_PACKAGES_IF_GPU="CUDA" # or CuArrays for older Julia versions
JULIA_NUM_THREADS=2
#---------------------------------------------------#

if [ -z `which julia` ]; then
  # Install Julia
  JULIA_VER=`cut -d '.' -f -2 <<< "$JULIA_VERSION"`
  echo "Installing Julia $JULIA_VERSION on the current Colab Runtime..."
  BASE_URL="https://julialang-s3.julialang.org/bin/linux/x64"
  URL="$BASE_URL/$JULIA_VER/julia-$JULIA_VERSION-linux-x86_64.tar.gz"
  wget -nv $URL -O /tmp/julia.tar.gz # -nv means "not verbose"
  tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
  rm /tmp/julia.tar.gz

  # Install Packages
  nvidia-smi -L &> /dev/null && export GPU=1 || export GPU=0
  if [ $GPU -eq 1 ]; then
    JULIA_PACKAGES="$JULIA_PACKAGES $JULIA_PACKAGES_IF_GPU"
  fi
  for PKG in `echo $JULIA_PACKAGES`; do
    echo "Installing Julia package $PKG..."
    julia -e 'using Pkg; pkg"add '$PKG'; precompile;"' &> /dev/null
  done

  # Install kernel and rename it to "julia"
  echo "Installing IJulia kernel..."
  julia -e 'using IJulia; IJulia.installkernel("julia", env=Dict(
      "JULIA_NUM_THREADS"=>"'"$JULIA_NUM_THREADS"'"))'
  KERNEL_DIR=`julia -e "using IJulia; print(IJulia.kerneldir())"`
  KERNEL_NAME=`ls -d "$KERNEL_DIR"/julia*`
  mv -f $KERNEL_NAME "$KERNEL_DIR"/julia

  echo ''
  echo "Successfully installed `julia -v`!"
  echo "Please reload this page (press Ctrl+R, ⌘+R, or the F5 key) then"
  echo "jump to the 'Checking the Installation' section."
fi

Installing Julia 1.10.4 on the current Colab Runtime...
2024-10-23 05:56:43 URL:https://julialang-s3.julialang.org/bin/linux/x64/1.10/julia-1.10.4-linux-x86_64.tar.gz [173704015/173704015] -> "/tmp/julia.tar.gz" [1]
Installing Julia package IJulia...
Installing Julia package BenchmarkTools...


# Checking the Installation
The `versioninfo()` function should print your Julia version and some other info about the system:

In [ ]:
versioninfo()

Julia Version 1.11.2
Commit 5e9a32e7af2 (2024-12-01 20:02 UTC)
Build Info:
  Official https://julialang.org/ release
Platform Info:
  OS: Linux (x86_64-linux-gnu)
  CPU: 2 × Intel(R) Xeon(R) CPU @ 2.20GHz
  WORD_SIZE: 64
  LLVM: libLLVM-16.0.6 (ORCJIT, broadwell)
Threads: 2 default, 0 interactive, 1 GC (on 2 virtual cores)
Environment:
  LD_LIBRARY_PATH = /usr/lib64-nvidia
  JULIA_NUM_THREADS = 2


# Need Help?

* Learning: https://julialang.org/learning/
* Documentation: https://docs.julialang.org/
* Questions & Discussions:
  * https://discourse.julialang.org/
  * http://julialang.slack.com/
  * https://stackoverflow.com/questions/tagged/julia

If you ever ask for help or file an issue about Julia, you should generally provide the output of `versioninfo()`.

Add new code cells by clicking the `+ Code` button (or _Insert_ > _Code cell_).

Have fun!

<img src="https://raw.githubusercontent.com/JuliaLang/julia-logo-graphics/master/images/julia-logo-mask.png" height="100" />

# Setup

Please run the following cell to setup the packages needed to follow along with the workshop. This can take some time to finish so it is recommended to do this ahead of time and allow it to finish.

In [1]:
import Pkg
Pkg.status()

Status `~/.julia/environments/v1.11/Project.toml`
  [6e4b80f9] BenchmarkTools v1.5.0
  [7073ff75] IJulia v1.26.0


In [ ]:
import Pkg
Pkg.add(["Flux", "NNlib", "CUDA", "Optimisers", "Transformers", "PromptingTools"])
Pkg.add(url = "https://github.com/cafaxo/Llama2.jl")
run(`wget 'https://huggingface.co/QuantFactory/Meta-Llama-3-8B-GGUF/resolve/main/Meta-Llama-3-8B.Q4_K_S.gguf?download=true -O Meta-Llama-3-8B.Q4_K_S.gguf'`)

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
   Installed OffsetArrays ──────────────── v1.14.1
   Installed LRUCache ──────────────────── v1.6.1
   Installed ShowCases ─────────────────── v0.1.0
   Installed ContextVariablesX ─────────── v0.1.3
   Installed LoggingExtras ─────────────── v1.1.0
   Installed LLVMLoopInfo ──────────────── v1.0.0
   Installed Accessors ─────────────────── v0.1.38
   Installed NNlib ─────────────────────── v0.9.26
   Installed ZipFile ───────────────────── v0.10.1
   Installed InlineStrings ─────────────── v1.4.2
   Installed StaticArrays ──────────────── v1.9.8
   Installed TimerOutputs ──────────────── v0.5.25
   Installed ConcurrentUtilities ───────── v2.4.2
   Installed HTML_Entities ─────────────── v1.0.1
   Installed Pidfile ───────────────────── v1.3.0
   Installed NVTX_jll ──────────────────── v3.1.0+2
   Installed Optimisers ────────────────── v0.3.4
   Installed CEnum ─────────────────────── v0.5.0


In [ ]:
using Flux, NNlib, CUDA, Optimisers, Transformers, Llama2

[ Info: Precompiling SpecialFunctionsExt [1285c0f1-ff9f-5867-b66e-0f359bcf09ba] (cache misses: wrong dep version loaded (2))
┌ Warning: Module CUDA with build ID fafbfcfd-b8ce-c588-0000-00a0f4bef89c is missing from the cache.
│ This may mean CUDA [052768ef-5323-5732-b1bb-66c8b64840ba] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:2541
┌ Info: Skipping precompilation due to precompilable error. Importing SpecialFunctionsExt [1285c0f1-ff9f-5867-b66e-0f359bcf09ba].
└   exception = Error when precompiling module, potentially caused by a __precompile__(false) declaration in the module.
[ Info: Precompiling ChainRulesCoreExt [eae2faf6-b232-58cb-a410-7764fda2830c] (cache misses: wrong dep version loaded (2))
┌ Warning: Module CUDA with build ID fafbfcfd-b8ce-c588-0000-00a0f4bef89c is missing from the cache.
│ This may mean CUDA [052768ef-5323-5732-b1bb-66c8b64840ba] does not support precompilation but is imported by a module that does.
└ @ Base loa

# What is Machine Learning in a nutshell

Machine Learning is the process of designing an algorithm which when shown some information or data can make _consistent_ inferences about the _patterns_ in the data

# Machine Learning is a Means to an End

We come up with tasks that we want to solve, and then design algorithms and experiments to meaningfully solve those tasks. Let us take text generation as an example.

In [ ]:
model = load_gguf_model("Meta-Llama-3-8B.Q4_K_S.gguf")

Loading model... 100%|███████████████████████████████████████████████████████| Time: 0:00:02


LanguageModel(
ModelConfig(
  dim            = 4096,
  hidden_dim     = 14336,
  n_layers       = 32,
  n_heads        = 32,
  n_kv_heads     = 8,
  vocab_size     = 128256,
  seq_len        = 512,
  rope_freq_base = 500000.0,
  rope_is_neox   = false,
))

In [ ]:
sample(model, "How is the weather today?")

 How is the weather today?  Sunny.
 What are you wearing today?  Blue jeans, red sweater, and brown boots.
 What kind of food are you in the mood for?  Burgers and fries.
 Do you want to go out for dinner today?  No.
 What would you like to do tonight?  Go to the movie.
-------
achieved tok/s: 0.46


# How do Machines Learn?

Many ways! Let us focus on one called Stochastic Gradient Descent. In simple words, we expose a "neural network" to some data, and ask it to perform a task. Let's say we ask it to identify numbers in a picture.

We then "grade" it comparing its output to the actual values. The function we use to do this grading is referred to as the _loss_ function. This can be interpreted as "how far is the model from what I expected it to output".

We now need to inform the model how to correct its output based on this distance. This is done by taking the gradient of the loss function.

The values of the gradient are then used to update the parameters of our model such that this distance is minimized. This is done with the help of an _optimiser_.

Several choices exist for each of these steps, and vary widely with the effects they have on the output. Our job is to piece together the relevant effects together.